# Análise do risco de inadimplência dos mutuários

O projeto consiste em descobrir se o estado civil de um cliente e o número de filhos têm impacto sobre se ele deixará de pagar um empréstimo. O banco já tem alguns dados sobre a capacidade de crédito dos clientes.

In [1]:
# Carregando todas as bibliotecas
import pandas as pd
import numpy as np
import seaborn as sns

# to ignore the warnings
from warnings import filterwarnings

# Carregando os dados
df = pd.read_csv("/datasets/credit_scoring_eng.csv")

In [2]:
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,19351.000000
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,26787.568355
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,16475.450632
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,3306.762000
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,16488.504500
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,23202.870000
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,32549.611000
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,362496.645000


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Exploração de dados

**Descrição dos dados**
- `children` - o número de crianças na família
- `days_employed` - experiência de trabalho em dias
- `dob_years` - idade do cliente em anos
- `education` - educação do cliente
- `education_id` - identificador de educação
- `family_status` - estado civil do cliente
- `family_status_id` - identificador de estado civil
- `gender` - gênero do cliente
- `income_type` - tipo de emprego
- `debt` - havia alguma dívida no pagamento do empréstimo
- `total_income` - renda mensal
- `purpose` - o objetivo de obter um empréstimo


In [4]:
df.shape

(21525, 12)

In [5]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding



Um dataframe com informaçoes acerca da vida finceira e pessoal de cada cliente. A principio observo que na coluna "days_employed" pode conter erros, na linha de index 4 temos um cliente com 340266.072047	empregado, o que totaliza 932 anos, evidenciado que este numero não faz sentido. Também dias negativos.

In [7]:
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Existem valores ausentes nas colunas "days_employed" e "total_income"

In [8]:
# Vejamos a tabela filtrada com valores ausentes na primeira coluna com dados ausentes
days_employed_null = df['days_employed'].isnull() 

df[days_employed_null].head(20)


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
65,0,NaN,21,secondary education,1,unmarried,4,M,business,0,NaN,transactions with commercial real estate
67,0,NaN,52,bachelor's degree,0,married,0,F,retiree,0,NaN,purchase of the house for my family
72,1,NaN,32,bachelor's degree,0,married,0,M,civil servant,0,NaN,transactions with commercial real estate
82,2,NaN,50,bachelor's degree,0,married,0,F,employee,0,NaN,housing
83,0,NaN,52,secondary education,1,married,0,M,employee,0,NaN,housing


In [9]:
df[days_employed_null].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2174 entries, 12 to 21510
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          2174 non-null   int64  
 1   days_employed     0 non-null      float64
 2   dob_years         2174 non-null   int64  
 3   education         2174 non-null   object 
 4   education_id      2174 non-null   int64  
 5   family_status     2174 non-null   object 
 6   family_status_id  2174 non-null   int64  
 7   gender            2174 non-null   object 
 8   income_type       2174 non-null   object 
 9   debt              2174 non-null   int64  
 10  total_income      0 non-null      float64
 11  purpose           2174 non-null   object 
dtypes: float64(2), int64(5), object(5)
memory usage: 220.8+ KB


Os valores ausentes não parecem simétricos. O que pude observar por cima é que parece que a maioria dos valorem ausentes compartilham da coluna 'debt' ser igual a zero.

In [10]:
# Vamos aplicar várias condições para filtrar dados e observar o número de linhas na tabela filtrada.
df.loc[ (days_employed_null) & (df['debt'] == 0) ]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
65,0,NaN,21,secondary education,1,unmarried,4,M,business,0,NaN,transactions with commercial real estate
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


In [11]:
df.loc[ (days_employed_null) & (df['education'] == "secondary education") ]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21423,0,NaN,63,secondary education,1,married,0,M,retiree,0,NaN,purchase of a car
21426,0,NaN,49,secondary education,1,married,0,F,employee,1,NaN,property
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


In [12]:
df.loc[ df['income_type'] == "retiree" ]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
18,0,400281.136913,53,secondary education,1,widow / widower,2,F,retiree,0,9091.804,buying a second-hand car
24,1,338551.952911,57,secondary education,1,unmarried,4,F,retiree,0,46487.558,transactions with commercial real estate
25,0,363548.489348,67,secondary education,1,married,0,M,retiree,0,8818.041,buy real estate
...,...,...,...,...,...,...,...,...,...,...,...,...
21505,0,338904.866406,53,secondary education,1,civil partnership,1,M,retiree,0,12070.399,to have a wedding
21508,0,386497.714078,62,secondary education,1,married,0,M,retiree,0,11622.175,property
21509,0,362161.054124,59,bachelor's degree,0,married,0,M,retiree,0,11684.650,real estate transactions
21518,0,373995.710838,59,SECONDARY EDUCATION,1,married,0,F,retiree,0,24618.344,purchase of a car


In [13]:
1408/2074

0.6788813886210222

In [14]:
2004/2074

0.9662487945998072

In [15]:
2174/21525

0.10099883855981417

O número de linhas filtradas correponde ao número de valores ausentes, talvez estas pessoas nunca tenham trabalhado por isso as colunas "days_employed" e "total_income" estao com valores ausentes.

A porcentagem de dados ausentes correspondem a 10% do total da amostra, uma quantidade consideravel em minha opinião.

68% das linhas com valores ausentes são de pessoas com a coluna 'education' igual a "secondary education". Pela porcentagem expressiva, acredito ter a ver com o motivo da ausencia de dados.

## Transformando dados

In [21]:
df.duplicated().sum()

54

In [22]:
df = df.drop_duplicates()

In [23]:
df.duplicated().sum()

0

In [24]:
# Vamos ver todos os valores na coluna de educação para verificar se e quais grafias precisarão ser corrigidas
df['education'].unique()

array(["bachelor's degree", 'secondary education', 'Secondary Education',
       'SECONDARY EDUCATION', "BACHELOR'S DEGREE", 'some college',
       'primary education', "Bachelor's Degree", 'SOME COLLEGE',
       'Some College', 'PRIMARY EDUCATION', 'Primary Education',
       'Graduate Degree', 'GRADUATE DEGREE', 'graduate degree'],
      dtype=object)

In [25]:
# Corrija os registros, se necessário
df['education'] = df['education'].str.lower()

In [26]:
# Verificando todos os valores na coluna para ter certeza de que os corrigimos
df['education'].unique()

array(["bachelor's degree", 'secondary education', 'some college',
       'primary education', 'graduate degree'], dtype=object)

In [27]:
# Vamos ver a distribuição de valores na coluna `children`
df['children'].value_counts()

 0     14107
 1      4809
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [28]:
df.loc[(df['children'] == -1)].info()
47*100/21471

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47 entries, 291 to 21140
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          47 non-null     int64  
 1   days_employed     44 non-null     float64
 2   dob_years         47 non-null     int64  
 3   education         47 non-null     object 
 4   education_id      47 non-null     int64  
 5   family_status     47 non-null     object 
 6   family_status_id  47 non-null     int64  
 7   gender            47 non-null     object 
 8   income_type       47 non-null     object 
 9   debt              47 non-null     int64  
 10  total_income      44 non-null     float64
 11  purpose           47 non-null     object 
dtypes: float64(2), int64(5), object(5)
memory usage: 4.8+ KB


0.2188999115085464

In [29]:
df.loc[(df['children'] == 20)].info()
76*100/21471

<class 'pandas.core.frame.DataFrame'>
Int64Index: 76 entries, 606 to 21491
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          76 non-null     int64  
 1   days_employed     67 non-null     float64
 2   dob_years         76 non-null     int64  
 3   education         76 non-null     object 
 4   education_id      76 non-null     int64  
 5   family_status     76 non-null     object 
 6   family_status_id  76 non-null     int64  
 7   gender            76 non-null     object 
 8   income_type       76 non-null     object 
 9   debt              76 non-null     int64  
 10  total_income      67 non-null     float64
 11  purpose           76 non-null     object 
dtypes: float64(2), int64(5), object(5)
memory usage: 7.7+ KB


0.35396581435424523

Existem 2 valores anormais na distrubuição de valores na coluna 'children'. São eles os valores '-1' e '20'. O primero porque não existem quantidade de filhos negativo e o segundo porque acho extremamente improvavel uma familia ter 20 filhos ainda mais 76 valores com este numero de filhos. A porcentagem que estes dados representam não chegam a 1% da amostra, portanto acredito que não afete o resultado da analise como um todo. Provavelmente estes 'erros' podem ter sido de digitação. Uma opção é utilizar a mediana para substitur estes valores.

In [30]:
test = df[(df['children'] != -1) & (df['children'] != 20)]
test['children'].median()

df['children'] = df['children'].replace(-1, 0)
df['children'] = df['children'].replace(20, 0)

In [31]:
# Verificar a coluna `children` novamente para ter certeza de que está tudo corrigido
df['children'].unique()

array([1, 0, 3, 2, 4, 5])

In [32]:
# Os dados probletmaticos em 'days_employed' sao os com valores NaN, negativos e os valores absurdos na asa dos 300mil dias trabalhoados.
df["days_employed"].describe()

count     19351.000000
mean      63046.497661
std      140827.311974
min      -18388.949901
25%       -2747.423625
50%       -1203.369529
75%        -291.095954
max      401755.400475
Name: days_employed, dtype: float64

In [33]:
df.loc[ df["days_employed"] > 0].sort_values(by="days_employed")

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
20444,0,328728.720605,72,secondary education,1,widow / widower,2,F,retiree,0,15443.094,purchase of the house for my family
9328,2,328734.923996,41,bachelor's degree,0,married,0,M,retiree,0,20319.600,transactions with my real estate
17782,0,328771.341387,56,secondary education,1,married,0,F,retiree,0,10983.688,transactions with commercial real estate
14783,0,328795.726728,62,bachelor's degree,0,married,0,F,retiree,0,12790.431,buying my own car
7229,1,328827.345667,32,secondary education,1,civil partnership,1,F,retiree,0,19546.075,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
7794,0,401663.850046,61,secondary education,1,civil partnership,1,F,retiree,0,7725.831,wedding ceremony
2156,0,401674.466633,60,secondary education,1,married,0,M,retiree,0,52063.316,cars
7664,1,401675.093434,61,secondary education,1,married,0,F,retiree,0,20194.323,housing transactions
10006,0,401715.811749,69,bachelor's degree,0,unmarried,4,F,retiree,0,9182.441,getting an education


In [34]:
df.loc[(days_employed_null)]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,secondary education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,bachelor's degree,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


In [35]:
df.loc[ df["days_employed"] < 0].describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,15906.000000,15906.000000,15906.000000,15906.000000,15906.000000,15906.000000,15906.000000
mean,0.552873,-2353.015932,39.818245,0.798378,0.969634,0.087326,27837.509634
std,0.788316,2304.243851,10.663171,0.554845,1.442263,0.282320,16980.846677
min,0.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,3418.824000
25%,0.000000,-3157.480084,32.000000,0.000000,0.000000,0.000000,17323.415000
50%,0.000000,-1630.019381,39.000000,1.000000,0.000000,0.000000,24181.535000
75%,1.000000,-756.371964,48.000000,1.000000,1.000000,0.000000,33839.106500
max,5.000000,-24.141633,75.000000,4.000000,4.000000,1.000000,362496.645000


In [36]:
df.loc[ (df["income_type"] != "retiree") & (df["days_employed"] > 0)]
## Aqui temos que dos valores absurdos, apenas 2 não correspondem ao income_type igual ' retiree'.

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
3133,1,337524.466835,31,secondary education,1,married,0,M,unemployed,1,9593.119,buying property for renting out
14798,0,395302.838654,45,bachelor's degree,0,civil partnership,1,F,unemployed,0,32435.602,housing renovation


In [37]:
# Removendo as 2 linhas com valores absurdos presentes no 'income_type' = "retiree", amostragem irrelevante.
df = df.drop([3133, 14798]).reset_index(drop=True)

In [38]:
# Transformando os valores negativos em absolutos.
df["days_employed"] = df["days_employed"].abs()

In [39]:
# Restante dos valores anormais em days_employed. Todos são de 'retiree'.
# Talvez a quantidade de dias trabalhos por uma pessoa aposentada não tenha importância.
# Pode-se levar isso em consideração na hora de fazer uma analise do crédito e deixar o valor como esta.

In [40]:
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21469.000000,19349.000000,21469.000000,21469.000000,21469.000000,21469.000000,19349.000000
mean,0.470958,66883.771346,43.279566,0.817225,0.973730,0.081047,26788.165099
std,0.750912,139004.404146,12.574592,0.548504,1.421133,0.272914,16475.788375
min,0.000000,24.141633,0.000000,0.000000,0.000000,0.000000,3306.762000
25%,0.000000,926.971650,33.000000,1.000000,0.000000,0.000000,16492.483000
50%,0.000000,2194.216968,42.000000,1.000000,0.000000,0.000000,23202.870000
75%,1.000000,5534.964957,53.000000,1.000000,1.000000,0.000000,32551.063000
max,5.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,362496.645000


In [41]:
df["days_employed"].describe()

count     19349.000000
mean      66883.771346
std      139004.404146
min          24.141633
25%         926.971650
50%        2194.216968
75%        5534.964957
max      401755.400475
Name: days_employed, dtype: float64

In [42]:
df["dob_years"].value_counts()

35    616
40    607
41    606
34    601
38    597
42    596
33    581
39    572
31    558
36    554
44    545
29    544
30    538
48    537
37    536
50    513
43    512
32    509
49    508
28    503
45    496
27    493
52    484
56    484
47    477
54    476
46    473
53    459
57    456
58    456
51    448
59    443
55    443
26    408
60    374
25    357
61    354
62    349
63    269
24    264
64    262
23    253
65    194
22    183
66    182
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

In [43]:
df["dob_years"].describe()

count    21469.000000
mean        43.279566
std         12.574592
min          0.000000
25%         33.000000
50%         42.000000
75%         53.000000
max         75.000000
Name: dob_years, dtype: float64

In [44]:
df.loc[ (df["dob_years"] == 0)]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
99,0,346541.618895,0,secondary education,1,married,0,F,retiree,0,11406.644,car
149,0,2664.273168,0,secondary education,1,divorced,3,F,employee,0,11228.230,housing transactions
270,3,1872.663186,0,secondary education,1,married,0,F,employee,0,16346.633,housing renovation
578,0,397856.565013,0,secondary education,1,married,0,F,retiree,0,15619.310,construction of own property
1040,0,1158.029561,0,bachelor's degree,0,divorced,3,F,business,0,48639.062,to own a car
...,...,...,...,...,...,...,...,...,...,...,...,...
19782,0,NaN,0,secondary education,1,married,0,F,employee,0,NaN,housing
20411,0,338734.868540,0,secondary education,1,married,0,F,retiree,0,41471.027,purchase of my own house
20526,0,331741.271455,0,secondary education,1,unmarried,4,F,retiree,0,20766.202,property
21125,2,108.967042,0,bachelor's degree,0,married,0,M,business,0,38512.321,building a real estate


Como a quantidade de linahs com idade igual a zero não representar uma parcela significativa da amostra, irei substituir estes valores pela mediana das idades.

In [45]:
df['dob_years'].median()

42.0

In [46]:
df['dob_years'] = df['dob_years'].replace(0, 42)

In [47]:
df["dob_years"].describe()

count    21469.000000
mean        43.477153
std         12.217880
min         19.000000
25%         34.000000
50%         42.000000
75%         53.000000
max         75.000000
Name: dob_years, dtype: float64

In [48]:
df["family_status"].value_counts()

married              12343
civil partnership     4162
unmarried             2810
divorced              1195
widow / widower        959
Name: family_status, dtype: int64

In [51]:
df["gender"].value_counts()

F      14188
M       7280
XNA        1
Name: gender, dtype: int64

In [52]:
df.loc[ (df["gender"] == "XNA")]
## Existe apenas um resultado estranho com o valor de 'XNA' como genero, como é 1 em 21k tanto faz mudar para M ou F.
df['gender'] = df['gender'].replace("XNA", "M")

In [53]:
df["gender"].unique()

array(['F', 'M'], dtype=object)

In [54]:
df["income_type"].value_counts()

employee                       11091
business                        5080
retiree                         3837
civil servant                   1457
entrepreneur                       2
paternity / maternity leave        1
student                            1
Name: income_type, dtype: int64

In [57]:
# Verificar duplicatas
df.duplicated().sum()

18

In [58]:
18*100/2496

0.7211538461538461

In [59]:
df[df.duplicated() == True]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
3288,0,NaN,58,secondary education,1,civil partnership,1,F,retiree,0,NaN,to have a wedding
6307,0,NaN,30,secondary education,1,married,0,M,employee,0,NaN,building a real estate
7915,0,NaN,64,bachelor's degree,0,civil partnership,1,F,retiree,0,NaN,having a wedding
7932,0,NaN,71,secondary education,1,civil partnership,1,F,retiree,0,NaN,having a wedding
9595,0,NaN,71,secondary education,1,civil partnership,1,F,retiree,0,NaN,having a wedding
9845,0,NaN,62,secondary education,1,married,0,F,retiree,0,NaN,to get a supplementary education
14074,0,NaN,48,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
14704,0,NaN,46,secondary education,1,civil partnership,1,F,employee,0,NaN,buying property for renting out
14986,0,NaN,42,secondary education,1,civil partnership,1,F,employee,0,NaN,having a wedding
15962,0,NaN,51,secondary education,1,civil partnership,1,F,business,0,NaN,having a wedding


In [60]:
# Removendo as duplicatas
df.drop_duplicates(inplace=True)

In [61]:
# Última verificação se temos duplicatas
df.duplicated().sum()

0

In [62]:
# Verificação do conjunto após os ajustes.
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21451 entries, 0 to 21468
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21451 non-null  int64  
 1   days_employed     19349 non-null  float64
 2   dob_years         21451 non-null  int64  
 3   education         21451 non-null  object 
 4   education_id      21451 non-null  int64  
 5   family_status     21451 non-null  object 
 6   family_status_id  21451 non-null  int64  
 7   gender            21451 non-null  object 
 8   income_type       21451 non-null  object 
 9   debt              21451 non-null  int64  
 10  total_income      19349 non-null  float64
 11  purpose           21451 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.1+ MB


O novo conjunto de dados tem as seguintes alterações do df original:
- Para a coluna 'education', os valores duplicados em decorrencia da grafia foram corrigidos.
- Para a coluna 'children', foram substituidos os valores anormais [-1, 20] por [0].
- Para a coluna 'days_employed, foram alterados os valores negativos para seu absoluto. Os valores absurdos x > 30000, foram mantidos pois todos são de pessoas aposentadas. Lembrar disso na hora de fazer analise dessa categoria.
- Para a coluna 'dob_years', foram substituidos os valores [0] pela mediana do conjunto.
- Para a coluna 'family_status', nada foi feito. Não foram encontrados erros.
- Para a coluna 'gender', foram alterados os valores anormais ["XNA"] por ["M"] por ser apenas uma linha. Poderia ter sido retirado do dataset também.
- Para a coluna 'income_type', nada foi feito. Não foram encontrados erros.

# Trabalhando com valores ausentes

In [63]:
# Encontre os dicionários
dict_days = {'days_employed':[df['days_employed'].mean(),df['days_employed'].median()]}
dict_income = {'total_income':[df['total_income'].mean(),df['total_income'].median()]}

In [64]:
dict_days

{'days_employed': [66883.77134582827, 2194.2169684631217]}

In [65]:
dict_income

{'total_income': [26788.165099488346, 23202.87]}

### Restaurar valores ausentes em `total_income`

As colunas 'total_income' e "days_employed' tem valores ausentos ambos nas mesmas linhas.

Irei iniciar agrupando por idade
- 19 a 35 - Jovem
- 35 a 60 - Adulto
- over 60 - Idoso

In [66]:
def category_age(row):
    if row <= 35:
        return 'young'
    if row <= 60:
        return 'adult'
    return 'elderly'    

In [67]:
# Testando se a função funciona
row_values = [66]
row_columns = ["dob_years"]
row = pd.Series(data=row_values, index=row_columns)
# passe a linha para a função e exiba a saída
print(category_age(row[0]))

elderly


In [68]:
# Criando coluna nova com base na função
df["category_by_age"] = df["dob_years"].apply(category_age)
df.tail()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,category_by_age
21464,1,4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions,adult
21465,0,343937.404131,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car,elderly
21466,1,2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property,adult
21467,3,3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car,adult
21468,2,1984.507589,40,secondary education,1,married,0,F,employee,0,13127.587,to buy a car,adult


In [69]:
# Verificando como os valores na nova coluna
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,category_by_age
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,adult
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,adult
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,young
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,young
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,adult


1 - Renda geralmente depende do nivel de educação da pessoa.

In [70]:
# Criando uma tabela sem valores ausentes e exibindo algumas de suas linhas para garantir que ela fique boa
df_not_null = df[df["days_employed"].isnull() == False]
df_not_null

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,category_by_age
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,adult
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,adult
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,young
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,young
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,adult
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21464,1,4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions,adult
21465,0,343937.404131,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car,elderly
21466,1,2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property,adult
21467,3,3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car,adult


In [71]:
df_not_null.groupby("education")["total_income"].describe()

,count,mean,std,min,25%,50%,75%,max
education,,,,,,,,
bachelor's degree,4715.0,33142.952424,21701.542134,5148.514,20254.12000,28043.3220,40015.8710,362496.645
graduate degree,6.0,27960.024667,12205.330046,15800.399,18005.02925,25161.5835,38593.8535,42945.794
primary education,261.0,21144.882211,10873.977874,4049.374,13117.13300,18741.9760,27119.0240,78410.774
secondary education,13692.0,24595.598668,13694.880025,3306.762,15622.59250,21837.2860,30224.8300,276204.162
some college,675.0,29045.443644,15633.692360,5514.581,18240.59300,25618.4640,36628.2880,153349.533


In [72]:
# valores médios de renda com base em seus fatores identificados
df_not_null.groupby("education")["total_income"].mean()

education
bachelor's degree      33142.952424
graduate degree        27960.024667
primary education      21144.882211
secondary education    24595.598668
some college           29045.443644
Name: total_income, dtype: float64

In [73]:
# valores medianos de renda com base em seus fatores identificados
df_not_null.groupby("education")["total_income"].median()

education
bachelor's degree      28043.3220
graduate degree        25161.5835
primary education      18741.9760
secondary education    21837.2860
some college           25618.4640
Name: total_income, dtype: float64

Utilizarei o valor mediano uma vez que o os valores maximo e minimos da renda diferem muito do valor mediano, contribuindo para que a média seja um valor não interessante para esta subtituição.

In [74]:
#função que usaremos para preencher os valores ausentes
def subs_total_income(row):
    total_income = row["total_income"]
    education = row["education"]
    if pd.isnull(total_income) == True:
        if education == "bachelor's degree":
            return 28054.5310
        if education == "graduate degree":
            return 25161.5835
        if education == "primary education":
            return 18741.9760
        if education == "secondary education":
            return 21836.5830
        if education == "some college":
            return 25618.4640
        else:
            pass
    else:
        return total_income        

In [75]:
row_values = ["2222", "primary education"] # valores de idade e desemprego
row_columns = ['total_income', 'education'] # nomes das colunas

row = pd.Series(data=row_values, index=row_columns) 
  
subs_total_income(row)

'2222'

In [76]:
# Aplicando em todas as linhas
df["total_income"] = df.apply(subs_total_income, axis=1)

In [77]:
# Verificando se temos algum erro
df[df["total_income"].isnull() == True]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,category_by_age


In [78]:
df["total_income"].describe()

count     21451.00000
mean      26466.88039
std       15701.73559
min        3306.76200
25%       17220.28250
50%       22584.95000
75%       31329.46550
max      362496.64500
Name: total_income, dtype: float64

In [80]:
df["total_income"]

0        40620.102
1        17932.802
2        23341.752
3        42820.568
4        25378.572
           ...    
21464    35966.698
21465    24959.969
21466    14347.610
21467    39054.888
21468    13127.587
Name: total_income, Length: 21451, dtype: float64

###  Restaurar valores em `days_employed`

In [82]:
df_not_null = df[df["days_employed"].isnull() == True]
df_not_null

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,category_by_age
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,21836.583,to have a wedding,elderly
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,21836.583,education,adult
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,21836.583,building a real estate,elderly
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,21836.583,second-hand car purchase,adult
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,21836.583,to have a wedding,adult
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21433,2,NaN,47,secondary education,1,married,0,M,business,0,21836.583,purchase of a car,adult
21439,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,21836.583,wedding ceremony,adult
21441,0,NaN,48,bachelor's degree,0,married,0,F,business,0,28054.531,building a property,adult
21446,1,NaN,42,secondary education,1,married,0,F,employee,0,21836.583,building a real estate,adult


In [83]:
# Distribuição de `days_employed` medianos com base em seus parâmetros identificados
df_not_null.groupby("category_by_age")["days_employed"].mean()

category_by_age
adult     NaN
elderly   NaN
young     NaN
Name: days_employed, dtype: float64

In [84]:
# Distribuição de `days_employed` médios com base em seus parâmetros identificados
df_not_null.groupby("category_by_age")["days_employed"].median()

category_by_age
adult     NaN
elderly   NaN
young     NaN
Name: days_employed, dtype: float64

Utilizarei a mediana porque os valores absurdos dos days_employed dos 'retiree' puxam muito a média pra cima.

In [85]:
def subs_days_employed(row):
    days_employed = row["days_employed"]
    age = row["dob_years"]
    if pd.isnull(days_employed) == True:   
        return ((age - 18) * 365)
    else:
        return days_employed

In [86]:
null_test = None
row_values1 = [null_test, 19] # valores de idade e desemprego
row_columns1 = ['days_employed', 'dob_years'] # nomes das colunas

row1 = pd.Series(data=row_values1, index=row_columns1) 
  
subs_days_employed(row1)

365.0

In [87]:
# Aplicando função ao days_employed
df["days_employed"] = df.apply(subs_days_employed, axis=1)

In [88]:
df_not_null = df[df["days_employed"].isnull() == True]
df_not_null

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,category_by_age


In [89]:
df["days_employed"]

0          8437.673028
1          4024.803754
2          5623.422610
3          4124.747207
4        340266.072047
             ...      
21464      4529.316663
21465    343937.404131
21466      2113.346888
21467      3112.481705
21468      1984.507589
Name: days_employed, Length: 21451, dtype: float64

## Categorização de dados

In [90]:
# valores dos dados selecionados para categorização
df[["family_status", "children", "purpose", "total_income"]]

,family_status,children,purpose,total_income
0,married,1,purchase of the house,40620.102
1,married,1,car purchase,17932.802
2,married,0,purchase of the house,23341.752
3,married,3,supplementary education,42820.568
4,civil partnership,0,to have a wedding,25378.572
...,...,...,...,...
21464,civil partnership,1,housing transactions,35966.698
21465,married,0,purchase of a car,24959.969
21466,civil partnership,1,property,14347.610
21467,married,3,buying my own car,39054.888


In [91]:
# valores exclusivos
df["family_status"].value_counts()

married              12338
civil partnership     4149
unmarried             2810
divorced              1195
widow / widower        959
Name: family_status, dtype: int64

In [92]:
df["children"].value_counts()

0    14212
1     4807
2     2052
3      330
4       41
5        9
Name: children, dtype: int64

- Os que tem um relacionamento e o que não tem [0, 1].
- Os que não tem filhos, os que tem entre 1-2 e os que tem 3+.


- Tem um relacionamento = 1, não tem = 0.
- 0 filhos, 1-2 filhos, 3+ filhos.

In [93]:
def cat_family_status(row):
    status = row["family_status"]
    if status in ["married", "civil partnership"]:
        return 1
    else:
        return 0
    
def cat_children_status(row):
    n = row["children"]
    if n == 0:
        return 0
    if n <= 2:
        return 1
    else:
        return 2

In [94]:
df["cat_family_status"] = df.apply(cat_family_status, axis=1)
df["cat_children_status"] = df.apply(cat_children_status, axis=1)

In [95]:
df["cat_family_status"].describe()

count    21451.000000
mean         0.768589
std          0.421744
min          0.000000
25%          1.000000
50%          1.000000
75%          1.000000
max          1.000000
Name: cat_family_status, dtype: float64

In [96]:
# Analisando o propósito do crédito
df["purpose"].value_counts()

wedding ceremony                            791
having a wedding                            767
to have a wedding                           765
real estate transactions                    675
buy commercial real estate                  661
housing transactions                        652
transactions with commercial real estate    650
buying property for renting out             650
housing                                     646
purchase of the house                       646
purchase of the house for my family         638
construction of own property                635
property                                    633
transactions with my real estate            627
building a real estate                      624
buy real estate                             621
purchase of my own house                    620
building a property                         619
buy residential real estate                 606
housing renovation                          606
buying my own car                       

In [141]:
house = ["wedding ceremony", "havin a wdding", "to have a wedding", "real estate transactions",
 "buy commercial real estate", "housing transactions", "transactions with commercial real estate",
"buying property for renting out", "housing", "purchase of the house", "purchase of the house for my family",
"construction of own property","property","transactions with my real estate", "buy real estate",
"building a real estate", "buy real state", "purchase of my own house", "building a property", "housing renovation",
"buy residential real estate"]
car = ["car","second-hand car purchase", "cars", "to own a car", "buying a second-hand car",
      "to buy a car", "car purchase", "purchase of a car", "buying my own car"]
education = ["supplementary education", "university education", "education", "to get a supplementary education",
             "getting an education", "profile education", "getting higher education", "to become educated",
            "going to university"]
wedding = ["wedding ceremony", "having a wedding", "to have a wedding"]

In [142]:
def cat_purpose(row):
    purpose = row["purpose"]
    if purpose in house:
        return "house"
    if purpose in car:
        return "car"
    if purpose in education:
        return "education"
    if purpose in wedding:
        return "wedding"
    else:
        return "ERROR"

In [144]:
df["cat_purpose"] = df.apply(cat_purpose, axis=1)

In [101]:
# Analisando a renda de cada cliente.
df["total_income"].describe()

count     21451.00000
mean      26466.88039
std       15701.73559
min        3306.76200
25%       17220.28250
50%       22584.95000
75%       31329.46550
max      362496.64500
Name: total_income, dtype: float64

In [172]:
def cat_total_income(row):
    total_income = row["total_income"]
    if total_income <= 5000:
        return "Menos que 5000"
    if total_income <= 15000:
        return "Menos que 15000"
    if total_income <= 25000:
        return "Menos que 25000"
    if total_income > 25000:
        return "Mais que 25000"

In [173]:
df["cat_total_income"] = df.apply(cat_total_income, axis=1)

In [174]:
df["cat_total_income"].value_counts()

Mais que 25000     9205
Menos que 25000    8504
Menos que 15000    3716
Menos que 5000       26
Name: cat_total_income, dtype: int64

In [175]:
df["cat_purpose"].value_counts()

house        12365
car           4306
education     4013
wedding        767
Name: cat_purpose, dtype: int64

## Verificar as Hipóteses


**Existe uma correlação entre o nível de renda e do pagamento em dia?**

In [187]:
# Correlação entre nivel de renda e pagamento em dia
pivot_renda = df.pivot_table(index= "cat_total_income", columns='debt', values = 'dob_years',
                       aggfunc = 'count', fill_value = 0)

# Calcular a taxa de inadimplência com base no número de filhos
pivot_renda['inad'] = round(100*pivot_renda[1]/(pivot_renda[0]+pivot_renda[1]),1)
pivot_renda['inad']

cat_total_income
Mais que 25000     7.6
Menos que 15000    7.9
Menos que 25000    8.7
Menos que 5000     7.7
Name: inad, dtype: float64

**Conclusão**

Não existe uma correlação clara entre o nivel de renda e o pagamento em dia.


**Existe uma correlação entre o status familiar e o pagamento em dia?**

In [193]:
# Correlação entre a quantidade de filhos e pagamento em dia
pivot_children = df.pivot_table(index= "cat_children_status", columns='debt', values = 'dob_years',
                       aggfunc = 'count', fill_value = 0)

# Calcular a taxa de inadimplência com base no número de filhos
pivot_children['inad'] = round(100*pivot_children[1]/(pivot_children[0]+pivot_children[1]),1)
pivot_children['inad']

cat_children_status
0    7.5
1    9.3
2    8.2
Name: inad, dtype: float64

In [200]:
df["cat_children_status"].value_counts()

0    14212
1     6859
2      380
Name: cat_children_status, dtype: int64

**Conclusão**

Aparentemente, existe uma correlação entre a quantidade de filhos e o pagamento em dia.
A porcentagem de pessoas que não tem filhos tomaradora de crédito que paga em dia é levemente maior de que quem tem 1 ou mais filhos.

**Existe uma correlação entre o status familiar e o pagamento em dia?**

In [202]:
# Correlação entre o status familiar e pagamento em dia
pivot_familiar = df.pivot_table(index= "cat_family_status", columns='debt', values = 'dob_years',
                       aggfunc = 'count', fill_value = 0)

# Calcular a taxa de inadimplência com status familiar
pivot_familiar['inad'] = round(100*pivot_familiar[1]/(pivot_familiar[0]+pivot_familiar[1]),1)
pivot_familiar['inad']

cat_family_status
0    8.5
1    8.0
Name: inad, dtype: float64

**Conclusão**

Não existe uma correlação clara entre o quantidade de filhos e o pagamento em dia.

**Como a finalidade do crédito afeta a taxa de inadimplência?**

In [201]:
# Confira os percentuais de inadimplência para cada finalidade de crédito e analise-os
pivot_purpose = df.pivot_table(index= "cat_purpose", columns='debt', values = 'dob_years',
                       aggfunc = 'count', fill_value = 0)

# Calcular a taxa de inadimplência com status familiar
pivot_purpose['inad'] = round(100*pivot_purpose[1]/(pivot_purpose[0]+pivot_purpose[1]),1)
pivot_purpose['inad']

cat_purpose
car          9.4
education    9.2
house        7.3
wedding      8.3
Name: inad, dtype: float64

**Conclusão**

Aparentemente, existe uma correlação entre a finalidade do crédito e o pagamento em dia.

Pessoas cujo propósito é comprar uma casa, aparenetemente são melhores pagadores do que pessoas cujo propósito é comprar um carro ou investir na educação.


# Conclusão Geral 
- Em uma análise que levou em conta apenas se a pessoa tem ou não uma familia, indepente do status em que se encontra a familia, este dado parece não influenciar na capacidade de pagamento de crédito do cliente.


- Em uma análise que levou em conta a quantidade de filhos do cliente, temos que pessoas sem filhos são melhores pagadores de crédito do que pessoas que tem filhos, sendo que o grupo que tem apenas 1 filho são os piores pagadores de crédito do que pessoas que não tem ou tem mais de 1.